In [24]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, Dense, Dropout, LSTM, GRU
from tensorflow.keras.models import load_model

In [17]:
# Load data
def json_to_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return pd.DataFrame(data)

set1_human = json_to_dataset("./data/set1_human.json")
set1_machine = json_to_dataset("./data/set1_machine.json")
set1_machine = set1_machine.drop('machine_id', axis=1)

set2_human = json_to_dataset("./data/set2_human.json")
set2_machine = json_to_dataset("./data/set2_machine.json")
set2_machine = set2_machine.drop('machine_id', axis=1)

print(f"set1 - human/machine : {len(set1_human)} / {len(set1_machine)}")
print(f"set2 - human/machine : {len(set2_human)} / {len(set2_machine)}")

# Label data and combine
set1_human["label"] = 1
set1_machine["label"] = 0
dataset1 = pd.concat([set1_human, set1_machine], ignore_index=True)

set2_human["label"] = 1
set2_machine["label"] = 0
dataset2 = pd.concat([set2_human, set2_machine], ignore_index=True)

set1 - human/machine : 122584 / 3500
set2 - human/machine : 100 / 400


In [18]:
# mock a local test set: 600 from domain 1 and 400 from domain 2.
# Domain 1 - 600 records
sample_human = dataset1[dataset1["label"] == 1].sample(300, random_state=42)
sample_machine = dataset1[dataset1["label"] == 0].sample(300, random_state=42)
# Combine the samples into a mock test dataset
mock_test_data1 = pd.concat([sample_human, sample_machine], ignore_index=True)

# Remove the samples from the original dataset
dataset1 = dataset1.drop(mock_test_data1.index)
# Preprocess the mock test dataset
X_mock_test = pad_sequences(mock_test_data1['txt'], maxlen=max_len)
y_mock_test = mock_test_data1["label"].values

# Domain 2 - 400 records
from sklearn.utils import resample
# Sample 100 records from the human class
sample_human = dataset2[dataset2["label"] == 1]
# Oversample the human records to reach 200 records
sample_human_oversampled = resample(sample_human, replace=True, n_samples=200, random_state=42)
# Sample 200 records from the machine class
sample_machine = dataset2[dataset2["label"] == 0].sample(200, random_state=42)
# Combine the samples into a mock test dataset
mock_test_data2 = pd.concat([sample_human_oversampled, sample_machine], ignore_index=True)
# Preprocess the mock test dataset
X_mock_test2 = pad_sequences(mock_test_data2['txt'], maxlen=max_len)
y_mock_test2 = mock_test_data2["label"].values

# Combine the two mock test sets
combined_mock_test = pd.concat([mock_test_data1, mock_test_data2], ignore_index=True)
X_combined_mock_test = pad_sequences(combined_mock_test['txt'], maxlen=max_len)
y_combined_mock_test = combined_mock_test["label"].values
print(f"generate {len(y_combined_mock_test)} mock tests.")

generate 1000 mock tests.


In [19]:
# Train/validation split
train_data1, val_data1 = train_test_split(dataset1, test_size=0.2, random_state=42, stratify=dataset1["label"])
train_data2, val_data2 = train_test_split(dataset2, test_size=0.2, random_state=42, stratify=dataset2["label"])

In [20]:
# fixed param
vocab_size = 5000

# model param
max_len = 250
embedding_dim = 64
lstm_unit = 64

# fit param
num_epoch = 50
num_batch = 64

In [22]:
strategy = "txt" # "prompt"
X_train1 = pad_sequences(train_data1[strategy], maxlen=max_len)
y_train1 = train_data1["label"].values
X_val1 = pad_sequences(val_data1[strategy], maxlen=max_len)
y_val1 = val_data1["label"].values

X_train2 = pad_sequences(train_data2[strategy], maxlen=max_len)
y_train2 = train_data2["label"].values
X_val2 = pad_sequences(val_data2[strategy], maxlen=max_len)
y_val2 = val_data2["label"].values

# Set up the EarlyStopping and ModelCheckpoint callback
checkpoint_filepath1 = './best_model1.h5'
checkpoint_filepath2 = './best_model2.h5'
model_checkpoint_callback1 = ModelCheckpoint(filepath=checkpoint_filepath1, save_best_only=True, 
                                             monitor='val_loss', mode='min')
model_checkpoint_callback2 = ModelCheckpoint(filepath=checkpoint_filepath2, save_best_only=True, 
                                             monitor='val_loss', mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
print("Preparation done.")

Preparation done.


In [26]:
# Define model - TRANSFER LEARNING
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(Bidirectional(GRU(lstm_unit, return_sequences=True)))
    model.add(Bidirectional(GRU(lstm_unit // 2)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# # Train model for domain 1
# model1 = create_model()

# print("Domain 1 model:")
# model1.fit(X_train1, y_train1, epochs=num_epoch, batch_size=num_batch, validation_data=(X_val1, y_val1), 
#            verbose=1, 
# #            class_weight=class_weights_dict, 
#            callbacks=[model_checkpoint_callback1, early_stopping])

print("-"*40)
print("Domain 2 model:")
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
best_val_loss = float('inf')
fold_accuracies = []

# Prepare the data for domain 2
X_train2 = pad_sequences(dataset2['txt'], maxlen=max_len)
y_train2 = dataset2["label"].values

# Transfer learning
model2 = load_model(checkpoint_filepath1)
model2.pop()

for layer in model2.layers[:-1]:
    layer.trainable = False

model2.add(Dense(1, activation='sigmoid'))
for i, (train_index, val_index) in enumerate(kf.split(X_train2)):
    print(f"\nFold {i + 1}/{num_folds}")
    X_train_fold, X_val_fold = X_train2[train_index], X_train2[val_index]
    y_train_fold, y_val_fold = y_train2[train_index], y_train2[val_index]

    model2.fit(X_train_fold, y_train_fold, epochs=num_epoch, batch_size=num_batch,
               validation_data=(X_val_fold, y_val_fold), verbose=1,
               callbacks=[model_checkpoint_callback2, early_stopping])
    model2 = load_model(checkpoint_filepath2)
    val_loss, _ = model2.evaluate(X_val_fold, y_val_fold, verbose=0)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model2 = model2
        print("best model updated!")

----------------------------------------
Domain 2 model:

Fold 1/3
Epoch 1/50
6/6 [==============================] - 6s 321ms/step - loss: 0.5829 - accuracy: 0.7327 - val_loss: 0.7110 - val_accuracy: 0.7246
Epoch 2/50
6/6 [==============================] - 0s 82ms/step - loss: 0.4801 - accuracy: 0.8378 - val_loss: 0.6602 - val_accuracy: 0.7246
Epoch 3/50
6/6 [==============================] - 0s 75ms/step - loss: 0.4616 - accuracy: 0.8348 - val_loss: 0.6358 - val_accuracy: 0.7246
Epoch 4/50
6/6 [==============================] - 0s 72ms/step - loss: 0.4597 - accuracy: 0.8348 - val_loss: 0.6551 - val_accuracy: 0.7246
Epoch 5/50
6/6 [==============================] - 0s 77ms/step - loss: 0.3994 - accuracy: 0.8468 - val_loss: 0.6066 - val_accuracy: 0.7246
Epoch 6/50
6/6 [==============================] - 0s 69ms/step - loss: 0.3919 - accuracy: 0.8529 - val_loss: 0.6531 - val_accuracy: 0.7305
Epoch 7/50
6/6 [==============================] - 0s 72ms/step - loss: 0.3493 - accuracy: 0.8498 -

In [30]:
# Load the best models
best_model1 = load_model(checkpoint_filepath1)
best_model2 = model2

# Make predictions and evaluate
y_pred_val1 = (best_model1.predict(X_val1) > 0.5).astype("int32")
y_pred_val2 = (best_model2.predict(X_train2) > 0.5).astype("int32")
print("\nValidation")
print("Domain1 Validation Accuracy:", accuracy_score(y_val1, y_pred_val1))
print("Domain2 Validation Accuracy:", accuracy_score(y_train2, y_pred_val2))
# mock test
# Split the combined mock test set into two parts: first 600 for model1 and next 400 for model2
X_mock_test1 = X_combined_mock_test[:600]
y_mock_test1 = y_combined_mock_test[:600]

X_mock_test2 = X_combined_mock_test[600:]
y_mock_test2 = y_combined_mock_test[600:]

# Make predictions using the respective models
y_pred_mock_test1 = (best_model1.predict(X_mock_test1) > 0.5).astype("int32")
y_pred_mock_test2 = (best_model2.predict(X_mock_test2) > 0.5).astype("int32")

# Calculate the accuracy for each part
accuracy_mock_test1 = accuracy_score(y_mock_test1, y_pred_mock_test1)
accuracy_mock_test2 = accuracy_score(y_mock_test2, y_pred_mock_test2)
print("-"*40)
print("Mock Test")
print("Mock Test1 Accuracy:", accuracy_mock_test1)
print("Mock Test2 Accuracy:", accuracy_mock_test2)

# Combine the predictions and true labels
y_pred_combined_mock_test = np.concatenate([y_pred_mock_test1, y_pred_mock_test2])
y_true_combined_mock_test = np.concatenate([y_mock_test1, y_mock_test2])

# Calculate the overall accuracy
accuracy_combined_mock_test = accuracy_score(y_true_combined_mock_test, y_pred_combined_mock_test)
print("Combined Mock Test Accuracy:", accuracy_combined_mock_test)

16/16 [==============================] - 0s 23ms/step

Validation
Domain1 Validation Accuracy: 0.983583695262382
Domain2 Validation Accuracy: 0.98
13/13 [==============================] - 0s 24ms/step
----------------------------------------
Mock Test
Mock Test1 Accuracy: 0.7833333333333333
Mock Test2 Accuracy: 0.945
Combined Mock Test Accuracy: 0.848


In [8]:
# Load test data
test_df = json_to_dataset("./data/test.json")

# Pad test sequences
X_test = pad_sequences(test_df['txt'], maxlen=max_len)

# Make predictions
X_test_domain1 = X_test[:600]
X_test_domain2 = X_test[600:]

predictions_domain1 = best_model1.predict(X_test_domain1)
predictions_domain2 = best_model2.predict(X_test_domain2)

# Combine predictions
predictions = np.concatenate([predictions_domain1, predictions_domain2])

# Convert predictions to binary labels
binary_predictions = (predictions > 0.5).astype(int).flatten()

# Create a submission DataFrame
submission_df = pd.DataFrame({"Id": np.arange(len(binary_predictions)), "Predicted": binary_predictions})

# Save the submission DataFrame as a CSV file
submission_df.to_csv("bidirection_transfer_cv.csv", index=False)
print("Done csv output.")

16/16 [==============================] - 1s 21ms/step

Domain1 Validation Accuracy: 0.9827893881111949
Domain2 Validation Accuracy: 1.0
13/13 [==============================] - 0s 20ms/step
Done csv output.
